In [3]:
from convertions import image_to_spectro

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
file_path = 'data/spectrograms/0.png'
spectro = image_to_spectro(file_path)

print(spectro.shape)

TypeError: image_to_spectro() missing 2 required positional arguments: 'min_spectro_db' and 'max_spectro_db'